# Plot histograms

N.B. Data should be placed in `/brainlit/data/axon_geometry/`

In [1]:
import numpy as np
import brainlit
import scipy
from brainlit.utils import swc
from cloudvolume.exceptions import SkeletonDecodeError
from brainlit.algorithms.trace_analysis.fit_spline import GeometricGraph
from brainlit.algorithms.trace_analysis.spline_fxns import curvature, torsion
import os
from pathlib import Path
import pandas as pd
from networkx.readwrite import json_graph
import json
import matplotlib.pyplot as plt
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "9"

device = torch.device("cuda:0" if torch.cuda.is_available() == True else "cpu")
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

/Users/thomasathey/Documents/mimlab/mouselight/env/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


## Define brain, find trace data folder

In [2]:
#specify brain1 or brain2 below
brain = "brain1"
bins = 2048
root_dir = data_dir = Path(os.path.abspath('')).parents[1]
data_dir = os.path.join(root_dir, "data/axon_geometry/{}".format(brain))
experiment_dir = os.path.join(root_dir, "experiments/axon_geometry")
segments_dir = os.path.join(data_dir, "segments")
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
print(f"Directory where swcs reside: {segments_swc_dir}")

Directory where swcs reside: /Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/data/axon_geometry/brain1/segments_swc


## Read trace data

In [3]:
max_id = 300
seg_lengths = []
mean_torsions = []
mean_curvatures = []
d_from_root = []
torsions = []
curvatures = []
for i in np.arange(0, max_id):
    i = int(i)
    trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
    if os.path.exists(trace_data_path) is True:
        trace_data = np.load(trace_data_path, allow_pickle=True)
        print("Loaded segment {}".format(i))

        for node in trace_data:
            # print(node)
            seg_lengths.append(node["seg_length"])
            mean_curvatures.append(node["mean_curvature"])
            mean_torsions.append(node["mean_torsion"])
            _curvatures = node["curvature"]
            _torsions = node["torsion"]
            assert len(_curvatures) == len(_torsions)
            m = len(_curvatures)
            # print(node["starting_length"])
            d_from_root.append(node["starting_length"] + np.arange(0, m))
            # d_from_root.append(np.arange(0, m)/node["seg_length"])
            torsions.append(_torsions)
            curvatures.append(_curvatures)
            
# ARRAY MANIPULATION TO CONVERT EVERYTHING TO NUMPY, REMOVE POINTS THAT WOULD YIELD -INF IN LOG
seg_lengths = np.array(seg_lengths)
mean_curvatures = np.array(mean_curvatures)
mean_torsions = np.array(mean_torsions)
d_from_root = np.concatenate(d_from_root)
curvatures = np.concatenate(curvatures)
torsions = np.concatenate(torsions)
# remove all root points
masked_d_from_root = np.ma.masked_less(d_from_root, 1e-16)
d_from_root = masked_d_from_root.compressed()
assert len(np.where(d_from_root < 1e-16)[0]) == 0
curvatures = curvatures[~masked_d_from_root.mask]
torsions = torsions[~masked_d_from_root.mask]

log_d_from_root = np.log10(d_from_root)
L = len(log_d_from_root)

Loaded segment 2
Loaded segment 7
Loaded segment 11
Loaded segment 12
Loaded segment 13
Loaded segment 14
Loaded segment 20
Loaded segment 24
Loaded segment 28
Loaded segment 35
Loaded segment 40
Loaded segment 45
Loaded segment 48
Loaded segment 49
Loaded segment 51
Loaded segment 53
Loaded segment 59
Loaded segment 60
Loaded segment 65
Loaded segment 67
Loaded segment 69
Loaded segment 71
Loaded segment 72
Loaded segment 73
Loaded segment 74
Loaded segment 75
Loaded segment 76
Loaded segment 77
Loaded segment 78
Loaded segment 79
Loaded segment 81
Loaded segment 82
Loaded segment 83
Loaded segment 85
Loaded segment 86
Loaded segment 91
Loaded segment 97
Loaded segment 98
Loaded segment 99
Loaded segment 101
Loaded segment 103
Loaded segment 106
Loaded segment 107
Loaded segment 109
Loaded segment 111
Loaded segment 112
Loaded segment 115
Loaded segment 118
Loaded segment 120
Loaded segment 124
Loaded segment 126
Loaded segment 127
Loaded segment 129
Loaded segment 132
Loaded segment 

## Compute histogram of the curvatures (uses GPU)

In [4]:
zero_curvatures_log_d_from_root = log_d_from_root[np.where(curvatures < 1e-16)[0]]
nonzero_curvatures_log_d_from_root = log_d_from_root[np.where(curvatures > 1e-16)[0]]
# compute histogram of zero-curvatures
torch_zero_curvatures_log_d_from_root = torch.from_numpy(zero_curvatures_log_d_from_root).to(device)
zero_curvatures_hist_min = torch.min(torch_zero_curvatures_log_d_from_root)
zero_curvatures_hist_max = torch.max(torch_zero_curvatures_log_d_from_root)
zero_curvatures_hist_bin_edges = torch.arange(
    zero_curvatures_hist_min,
    zero_curvatures_hist_max,
    (zero_curvatures_hist_max - zero_curvatures_hist_min)/bins)

zero_curvatures_hist = torch.histc(
    torch_zero_curvatures_log_d_from_root,
    bins=bins,
    min=zero_curvatures_hist_min,
    max=zero_curvatures_hist_max,
)
# compute histogram of non-zero-curvatures (uses GPU)
torch_nonzero_curvatures_log_d_from_root = torch.from_numpy(nonzero_curvatures_log_d_from_root).to(device)
nonzero_curvatures_hist_min = torch.min(torch_nonzero_curvatures_log_d_from_root)
nonzero_curvatures_hist_max = torch.max(torch_nonzero_curvatures_log_d_from_root)
nonzero_curvatures_hist_bin_edges = torch.arange(
    nonzero_curvatures_hist_min,
    nonzero_curvatures_hist_max,
    (nonzero_curvatures_hist_max - nonzero_curvatures_hist_min) / bins,
)
nonzero_curvatures_hist = torch.histc(
    torch_nonzero_curvatures_log_d_from_root,
    bins=bins,
    min=nonzero_curvatures_hist_min,
    max=nonzero_curvatures_hist_max
)

alpha_zero_curvatures = len(zero_curvatures_log_d_from_root) / L
alpha_nonzero_curvatures = len(nonzero_curvatures_log_d_from_root) / L
print(alpha_zero_curvatures, alpha_nonzero_curvatures)

0.016040546451364344 0.9839594535486357


## Compute histogram of the torsions (uses GPU)

In [5]:
zero_torsions_log_d_from_root = log_d_from_root[np.where(torsions < 1e-16)[0]]
nonzero_torsions_log_d_from_root = log_d_from_root[np.where(torsions > 1e-16)[0]]
# compute histogram of zero-torsions
torch_zero_torsions_log_d_from_root = torch.from_numpy(zero_torsions_log_d_from_root).to(device)
zero_torsions_hist_min = torch.min(torch_zero_torsions_log_d_from_root)
zero_torsions_hist_max = torch.max(torch_zero_torsions_log_d_from_root)
zero_torsions_hist_bin_edges = torch.arange(
    zero_torsions_hist_min,
    zero_torsions_hist_max,
    (zero_torsions_hist_max - zero_torsions_hist_min)/bins)

zero_torsions_hist = torch.histc(
    torch_zero_torsions_log_d_from_root,
    bins=bins,
    min=zero_torsions_hist_min,
    max=zero_torsions_hist_max,
)
# compute histogram of non-zero-torsions (uses GPU)
torch_nonzero_torsions_log_d_from_root = torch.from_numpy(nonzero_torsions_log_d_from_root).to(device)
nonzero_torsions_hist_min = torch.min(torch_nonzero_torsions_log_d_from_root)
nonzero_torsions_hist_max = torch.max(torch_nonzero_torsions_log_d_from_root)
nonzero_torsions_hist_bin_edges = torch.arange(
    nonzero_torsions_hist_min,
    nonzero_torsions_hist_max,
    (nonzero_torsions_hist_max - nonzero_torsions_hist_min) / bins,
)
nonzero_torsions_hist = torch.histc(
    torch_nonzero_torsions_log_d_from_root,
    bins=bins,
    min=nonzero_torsions_hist_min,
    max=nonzero_torsions_hist_max
)

alpha_zero_torsions = len(zero_torsions_log_d_from_root) / L
alpha_nonzero_torsions = len(nonzero_torsions_log_d_from_root) / L
print(alpha_zero_torsions, alpha_nonzero_torsions)

0.5134358033856311 0.4865641966143689


## Plot figure

Text(0.5, 0.98, 'Brain 1')

In [6]:
fig = plt.figure(figsize=(21, 7))
axes = fig.subplots(1, 2)
GRAY = "#999999"
TITLE_TYPE_SETTINGS = {"fontname": "Arial", "size": 20}
SUP_TITLE_TYPE_SETTINGS = {"fontname": "Arial", "size": 24}
plt.rc("font", family="Arial", size=20)

ax = axes[0]
ax.spines["bottom"].set_color(GRAY)
ax.spines["top"].set_color(GRAY)
ax.spines["right"].set_color(GRAY)
ax.spines["left"].set_color(GRAY)
ax.tick_params(axis="both", colors=GRAY, labelsize="large")

ax.bar(zero_curvatures_hist_bin_edges.cpu().numpy(), zero_curvatures_hist.cpu().numpy() / L, label=r"$\mathcal{k} = 0$", alpha=0.5)
ax.bar(nonzero_curvatures_hist_bin_edges.cpu().numpy(), nonzero_curvatures_hist.cpu().numpy() / L, label=r"$\mathcal{k} > 0$", alpha=0.5)

ax.set_title(r"Curvature ($\alpha = %.4f$)" % alpha_zero_curvatures)
ax.set_xlabel(r"$\log$ distance from root ($\mu m$)", fontsize=24)
ax.set_ylabel(r"pdf", fontsize=24)
leg = ax.legend(loc=2)
leg.get_frame().set_edgecolor(GRAY)

ax = axes[1]
ax.spines["bottom"].set_color(GRAY)
ax.spines["top"].set_color(GRAY)
ax.spines["right"].set_color(GRAY)
ax.spines["left"].set_color(GRAY)
ax.tick_params(axis="both", colors=GRAY, labelsize="large")

ax.bar(zero_torsions_hist_bin_edges.cpu().numpy(), zero_torsions_hist.cpu().numpy() / L, label=r"$\tau = 0$", alpha=0.5)
ax.bar(nonzero_torsions_hist_bin_edges.cpu().numpy(), nonzero_torsions_hist.cpu().numpy() / L, label=r"$\tau > 0$", alpha=0.5)

ax.set_title(r"Torsion ($\alpha = %.4f$)" % alpha_zero_torsions)
ax.set_xlabel(r"$\log$ distance from root ($\mu m$)", fontsize=24)
ax.set_ylabel(r"pdf", fontsize=24)
leg = ax.legend(loc=2)
leg.get_frame().set_edgecolor(GRAY)

title = "Brain 1" if brain == "brain1" else "Brain 2"
fig.suptitle(title)

## Save figure

In [7]:
plt.savefig(os.path.join(experiment_dir, "figures/{}_histograms_from_root.jpg".format(brain)))
plt.savefig(os.path.join(experiment_dir, "figures/{}_histograms_from_root.eps".format(brain)))

<Figure size 432x288 with 0 Axes>